In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Grid
import pandas as pd
import akshare as ak
import pickle
from utils.log import logger
from utils.fetch_stock_data import get_data_from_all
# 读取temp_data/stock_data_20210701_20230328.pkl
with open('temp_data/stock_data_20210701_20230328.pkl', 'rb') as f:
    stock_data = pickle.load(f)

In [60]:


# 读取股票数据
df = stock_data[('600519', '贵州茅台')]


def kline_compare_base(stock_df):
    # 将数据转换为列表格式
    data = stock_df[['open', 'close', 'low', 'high']].values.tolist()
    dates = stock_df.index.tolist() # date type:str, like '2021-07-01'

    # 绘制 K 线图
    kline = (
        Kline(init_opts=opts.InitOpts(bg_color="#000000"))

        .set_global_opts(
            title_opts=opts.TitleOpts(title="双品种组合"),
            xaxis_opts=opts.AxisOpts(
                type_='category',
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)
            ),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitline_opts=opts.SplitLineOpts(is_show=False)
            ),
            legend_opts=opts.LegendOpts(
                is_show=True, textstyle_opts=opts.TextStyleOpts(color='#DCDCDC'))
        )

        .add_xaxis(dates)
        .add_yaxis(
            "K线图",
            data,
            itemstyle_opts=opts.ItemStyleOpts(
                color="#ec0000",
                color0="#00da3c",
                border_color="#8A0000",
                border_color0="#008F28",
            ),
        )
    )

    # 绘制均线图
    # 计算均线数据
    ma5 = stock_df['close'].rolling(window=5).mean().tolist()
    ma10 = stock_df['close'].rolling(window=10).mean().tolist()
    ma20 = stock_df['close'].rolling(window=20).mean().tolist()


    line = (
        Line()

        .set_global_opts(
            title_opts=opts.TitleOpts(title="双品种组合1"),
            
        )

        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),  # 去掉line上的标签
        )
        .add_xaxis(dates)
        .add_yaxis("MA5", ma5, symbol='none')  # symbol='none'去掉line上的圆点
        .add_yaxis("MA10", ma10, symbol='none')
        .add_yaxis("MA20", ma20, symbol='none')
    )

    # 将 K 线图和均线图组合在一起
    k_ma  = kline.overlap(line)
    return  k_ma


In [61]:
def kline_compare(df_top,df_bottom,start_value,end_value):
    kline_top = kline_compare_base(df_top)
    kline_bottom = kline_compare_base(df_bottom)
    kline_top.set_global_opts(datazoom_opts=[
        opts.DataZoomOpts(is_show=True,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=0, pos_top="94%",pos_bottom="4%"), 
        opts.DataZoomOpts(is_show=True,start_value=start_value, end_value=end_value, range_start=None,range_end=None, xaxis_index=1, pos_top="97%", pos_bottom="1%"),
        ]
    )
    
    grid = Grid(init_opts=opts.InitOpts(width="1600px", height="800px"))
    
    grid.add(kline_top, grid_opts=opts.GridOpts(pos_top="5%", pos_bottom="50%"))
    grid.add(kline_bottom, grid_opts=opts.GridOpts(pos_top="55%", pos_bottom="10%"))

    return grid


# df_top=stock_data[('600519', '贵州茅台')]
df_top=get_data_from_all('600519',stock_data)
df_bottom = stock_data[('600172', '黄河旋风')]
kline_compare(df_top,df_bottom,'2021-01-12','2021-07-21').render_notebook()

In [ ]:
import random
x_data = [str(i) for i in list(range(1990, 2020))]

y_data_1 = [random.randint(0, 100) for _ in x_data]
y_data_2 = [random.randint(0, 100) for _ in x_data]


line1 = (
    Line()
    .add_xaxis(x_data)
    .add_yaxis('图例1', y_data_1)
    # .set_global_opts(datazoom_opts=[opts.DataZoomOpts(range_start=50, range_end=80,xaxis_index=0,pos_top="80%"),opts.DataZoomOpts(range_start=50, range_end=80,xaxis_index=1,pos_top="90%")])
    .set_global_opts(datazoom_opts=[opts.DataZoomOpts(start_value=1995, end_value=2005, xaxis_index=0, pos_top="80%"), opts.DataZoomOpts(start_value=1995, end_value=2005, xaxis_index=1, pos_top="90%")])
)


line2 = (
    Line()
    .add_xaxis(x_data)
    .add_yaxis('图例2', y_data_2)

)

grid = Grid()
grid.add(line1, grid_opts=opts.GridOpts(pos_top="10%", pos_bottom="60%"))
grid.add(line2, grid_opts=opts.GridOpts(pos_top="60%", pos_bottom="20%"))


# Render the chart
grid.render_notebook()

In [38]:
from pyecharts.charts import Line
from pyecharts import options as opts

c = (
    Line()
    .add_xaxis(["2019/04/01", "2019/04/02", "2019/04/03", "2019/04/04"])
    .add_yaxis(
        "",
        [5, 20, 36, 2],
        is_symbol_show=False,
        markline_opts=opts.MarkLineOpts(
            data=[opts.MarkLineItem(y=2)],
            # 去掉参考线箭头
            symbol="none",
        ),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="time"),
        tooltip_opts=opts.TooltipOpts(
            trigger="axis", axis_pointer_type="cross"),
        datazoom_opts=opts.DataZoomOpts(
            is_show=True,
            start_value="2019/04/03",
            range_start=None,
            range_end=None,
            pos_bottom=-2,
        ),
    )
)
c.render_notebook()